In [ ]:
###---Some general imports---###
import numpy as np
from scipy.stats import norm

from glob import glob
from astropy.io import ascii
import bessels

# Performing some initial precomputations

In [ ]:
# This snippet rearranges the EFTCAMB output files for ease of use with the Fortran integrator.
# Download the required files from https://www.dropbox.com/sh/m1lrzdjxwp3ioyl/AADRHWZxE-Fn-Un-HBApEYxja?dl=0 
# and unzip the contents in the /Data folder
# Go grab a coffee, this might take about an hour!

###---IN CASE YOU ARE IN A HURRY---###
# We also provide the output of this snippet. 
# In case you would like to use them, simply download them from 
# https://www.dropbox.com/sh/to40ws3amzammb5/AACAGU7P03fP473uXfxyKlfua?dl=0
# Unzip the contents into /Data/grid folder.
# Skip this snippet altogether.



ell_lst = np.arange(10, 100)

for counter, filename in enumerate(glob('../Data/EFT_CAMB_output/*.dat')):
    print counter, filename 
    
    newfilename =  filename.strip('../Data/EFT_CAMB_output/').strip('_cache_MetricMG.dat')
    Fortran_output = ascii.read(filename)
    #Let's cut out the very large z's.
    Fortran_output = Fortran_output[(1./Fortran_output['a'].data - 1.) < 7.]

    #Separate columns
    a_lst = Fortran_output['a'].data
    eta_lst = Fortran_output['tau'].data
    k_lst = Fortran_output['k'].data
    confH_lst = Fortran_output['adotoa'].data
    sigma_lst = Fortran_output['sigma'].data
    deltaCDM_lst = Fortran_output['clxc'].data

    #The unique values of k and eta
    k_unique_lst = np.unique(k_lst)
    eta_unique_lst = np.unique(eta_lst)

    #The number of unique 'k' and 'eta' values
    k_length = len(k_unique_lst)
    eta_length = len(eta_unique_lst)

    #Note that the number of unique 'a' values is not the same as the num of 'eta' unique values 
    a_unique_lst = [ a_lst[np.where(eta_lst == eta_unique)[0][0]] for eta_unique in eta_unique_lst]


    #Finally, let's reshape in the form of k_length x eta_length
    confH = np.reshape(confH_lst, (k_length, eta_length))
    sigma = np.reshape(sigma_lst, (k_length, eta_length))
    deltaCDM = np.reshape(deltaCDM_lst, (k_length, eta_length))

    #We define this array for using in Bessel function calculations
    keta_lst = np.kron(k_unique_lst, (max(eta_unique_lst) - eta_unique_lst))

    num_x = 100000
    max_x = max(k_unique_lst)*(max(eta_unique_lst) - min(eta_unique_lst))
    x = np.linspace(0., max_x, num_x)

    k_Recompute_index = 50
    ell_Recompute = len(ell_lst)

    max_x_Recompute = k_unique_lst[k_Recompute_index]*(max(eta_unique_lst) - min(eta_unique_lst))
    x_Recompute = np.linspace(0., max_x_Recompute, num_x)


    bess_int_keta_lst, bessPrime_int_keta_lst = \
        bessels.bessel_calculation(1, ell_lst, x, x_Recompute, ell_Recompute, keta_lst, k_Recompute_index)

    bess_int_keta_reshaped = np.reshape(bess_int_keta_lst, (len(ell_lst), k_length, eta_length))
    bessPrime_int_keta_reshaped = np.reshape(bessPrime_int_keta_lst, (len(ell_lst), k_length, eta_length))


    np.save('../Data/grid/F/'+newfilename+'.npy', [a_unique_lst, k_unique_lst, eta_unique_lst, \
                                              bess_int_keta_reshaped, confH, sigma, \
                                              deltaCDM, k_length, eta_length])



In [ ]:
# This snippet rearranges the EFTCAMB output files for ease of use with the Python integrator.
###---The Python integrator is completely equivalent to the Fortran integrator in terms of accuracy.
###---It is, however, somewhat slower.
# Download the required files from https://www.dropbox.com/sh/m1lrzdjxwp3ioyl/AADRHWZxE-Fn-Un-HBApEYxja?dl=0 
# Unzip the contents in the /Data folder.
# Go grab a coffee, this might take about an hour!

from glob import glob
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt
from scipy.special import spherical_jn
from scipy.interpolate import interp1d
from scipy.integrate import quad, simps, trapz
from math import factorial
from time import time

for counter, filename in enumerate(glob('../Data/EFT_CAMB_output/*.dat')):
        
    print counter, filename
    newfilename =  filename.strip('../Data/EFT_CAMB_output/').strip('_cache_MetricMG.dat')
    
    
    data = Table.read(filename, format="ascii")
    
    
    # Interpolation points for the k*dtau 
    N_points = 200000

    # How many l's?
    l_list = np.arange(10, 100)


    # unique k, tau arrays
    k_unique = np.unique(data['k'])
    tau_unique = np.unique(data['tau'])
    tau_max = tau_unique.max()

    N_tau = len(tau_unique)
    N_k = len(k_unique)
    N_l = len(l_list)



    x_int_max = k_unique[-1]*tau_unique[-1]*1.1


    l_list_small = l_list[l_list<5]
    N_l_small = (l_list<5).sum()

    print "N_tau, N_k, N_l are", N_tau, N_k, N_l    
    
    k = np.tile(data['k'].quantity.value.reshape(N_k, N_tau), (N_l, 1, 1))
    tau = np.tile(data['tau'].quantity.value.reshape(N_k, N_tau), (N_l, 1, 1))
    H = np.tile(data['adotoa'].quantity.value.reshape(N_k, N_tau), (N_l, 1, 1))
    clxc = np.tile(data['clxc'].quantity.value.reshape(N_k, N_tau), (N_l, 1, 1))
    sigma = np.tile(data['sigma'].quantity.value.reshape(N_k, N_tau), (N_l, 1, 1))
    

    j_int = np.zeros((N_l, N_tau*N_k))
    j_p_int = np.zeros((N_l, N_tau*N_k))
    x_int = np.linspace(0, x_int_max, N_points)
    dx = x_int[2]-x_int[1]    
    
    
    
    for i in xrange(N_l):
        y_int = spherical_jn(l_list[i], x_int)
        #y_int_c = (y_int[1:]+y_int[:-1])*(x_int[1:] - x_int[:-1])/2. 
        y_int_c = np.cumsum(y_int)*dx
        j_int[i, :] =  -np.interp( (data['k']*(tau_max-data['tau'])).quantity.value, x_int, y_int_c)/data['k'].quantity.value
        j_p_int[i, :] = -np.interp( (data['k']*(tau_max-data['tau'])).quantity.value, x_int, y_int)/data['k'].quantity.value

    j_int_old = np.copy(j_int)
    j_int_old = np.reshape(j_int_old, (N_l, N_k, N_tau))
    j_int_old = np.gradient(j_int_old, axis=2)


    print "super grid"
    x_sl = (data['k']*(tau_max-data['tau'])).quantity.value
    k_sl = data['k'].quantity.value

    x_int_max_2 = x_int_max/N_points*1000.
    x_int_2 = np.linspace(0, x_int_max_2, N_points)
    dx =  x_int_2[2]-x_int_2[1]

    idx = x_sl < x_int_max_2

    for i in xrange(N_l_small):
        y_int = spherical_jn(l_list_small[i], x_int_2) 
        y_int_c = np.cumsum(y_int)*dx
        j_int[i, idx] =  -np.interp( x_sl[idx], x_int_2, y_int_c)/k_sl[idx]
        j_p_int[i, idx] = -np.interp( x_sl[idx], x_int_2, y_int)/k_sl[idx]


    j_int = np.reshape(j_int, (N_l, N_k, N_tau))
    j_p_int = np.reshape(j_p_int, (N_l, N_k, N_tau))
    j_int_temp = np.diff(j_int, axis=2)
    j_p_int_temp = np.diff(j_p_int, axis=2)

    j_int[:, :, :-1] = j_int_temp
    j_p_int[:, :, :-1] = j_p_int_temp

    j_int[:, :, -1] = 0
    j_p_int[:, :, -1] = 0
    
    
    a_unique = data['a'].quantity.value.reshape(N_k, N_tau)[0]
    np.save('../Data/grid/P/'+newfilename+'.npy', [a_unique, data['clxc'].quantity.value, j_int, data['adotoa'].quantity.value, data['sigma'].quantity.value, data['k'].quantity.value, N_k, N_l, N_tau])
    
    
    

# Performing the necessary precomputations for a range of $k_\mathrm{max}$ values, for $\Omega_\mathrm{m} \approx 0.32$

In [ ]:
ell_lst = np.arange(10, 500)

Om_baryons = 0.0493
num_Om_M = 60
Om_M_lst = np.round(np.linspace(0.1, 0.7, num_Om_M), 4)
Om_indx = np.abs(0.32 - Om_M_lst).argmin() #Om approx 0.32

filename = '../Data/EFT_CAMB_output/' + str(Om_indx) + '_cache_MetricMG.dat'
Fortran_output = ascii.read(filename)

#print np.max(Fortran_output['k'].data)

k_max_lst = np.arange(0.32, 0.08, -0.02)

for indx, k_max_tmp in enumerate(k_max_lst):
    
    print indx + 1, k_max_tmp
    
    #Let's cut out the very large z's.
    Fortran_output = Fortran_output[(1./Fortran_output['a'].data - 1.) < 7.]
    
    #Let's cut out the k's.
    Fortran_output = Fortran_output[Fortran_output['k'].data < k_max_tmp]
    
    #Separate columns
    a_lst = Fortran_output['a'].data
    eta_lst = Fortran_output['tau'].data
    k_lst = Fortran_output['k'].data
    confH_lst = Fortran_output['adotoa'].data
    sigma_lst = Fortran_output['sigma'].data
    deltaCDM_lst = Fortran_output['clxc'].data
    
    #The unique values of k and eta
    k_unique_lst = np.unique(k_lst)
    eta_unique_lst = np.unique(eta_lst)

    #The number of unique 'k' and 'eta' values
    k_length = len(k_unique_lst)
    eta_length = len(eta_unique_lst)

    #Note that the number of unique 'a' values is not the same as the num of 'eta' unique values 
    a_unique_lst = [ a_lst[np.where(eta_lst == eta_unique)[0][0]] for eta_unique in eta_unique_lst]


    #Finally, let's reshape in the form of k_length x eta_length
    confH = np.reshape(confH_lst, (k_length, eta_length))
    sigma = np.reshape(sigma_lst, (k_length, eta_length))
    deltaCDM = np.reshape(deltaCDM_lst, (k_length, eta_length))

    #We define this array for using in Bessel function calculations
    keta_lst = np.kron(k_unique_lst, (max(eta_unique_lst) - eta_unique_lst))

    num_x = 100000
    max_x = max(k_unique_lst)*(max(eta_unique_lst) - min(eta_unique_lst))
    x = np.linspace(0., max_x, num_x)

    k_Recompute_index = 50
    ell_Recompute = len(ell_lst)

    max_x_Recompute = k_unique_lst[k_Recompute_index]*(max(eta_unique_lst) - min(eta_unique_lst))
    x_Recompute = np.linspace(0., max_x_Recompute, num_x)


    bess_int_keta_lst, bessPrime_int_keta_lst = \
        bessels.bessel_calculation(1, ell_lst, x, x_Recompute, ell_Recompute, keta_lst, k_Recompute_index)

    bess_int_keta_reshaped = np.reshape(bess_int_keta_lst, (len(ell_lst), k_length, eta_length))
    bessPrime_int_keta_reshaped = np.reshape(bessPrime_int_keta_lst, (len(ell_lst), k_length, eta_length))


    np.save('../Data/grid/F/k_range_tests/' + str(indx + 1) + '.npy', [a_unique_lst, k_unique_lst, eta_unique_lst, \
                                              bess_int_keta_reshaped, confH, sigma, \
                                              deltaCDM, k_length, eta_length])

